In [23]:
from transformers import BertConfig, BertModel, AutoTokenizer, TrainingArguments, AutoModelForSequenceClassification, Trainer, DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import torch
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict, ClassLabel
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

In [24]:
torch.cuda.is_available()

True

In [25]:
tokenizer = AutoTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")

In [26]:
# Loading dataset
dataSetPath = "2011-2018-3.csv"
df = pd.read_csv(dataSetPath, sep="␞", nrows=2500)
# df = pd.read_csv(dataSetPath, sep="␞", nrows=10000)


print(df['text'].isna().sum())
df = df.dropna(subset=['text'])
print(len(df))

print(df.columns)
df = df.drop(['id', 'url', 'headline', 'datePublished'], axis=1) 

def multiLabelToSingular(labelArray):
    cleaned_text = labelArray.replace("'", "").replace(" ", "").replace("[", "").replace("]", "")
    return cleaned_text.split(",")[0]

print(df.dtypes)
df['subjects'] = df['subjects'].apply(multiLabelToSingular)
print(df['subjects'].nunique())
print(df['subjects'].unique().tolist())

df.rename({'subjects': 'labels'}, axis="columns", inplace=True)

le = preprocessing.LabelEncoder()
le.fit(df.labels)
df['labels'] = le.transform(df.labels)

4
2496
Index(['id', 'url', 'headline', 'text', 'subjects', 'datePublished'], dtype='object')
text        object
subjects    object
dtype: object
11
['urheilu', 'ulkomaat', 'kotimaa', 'kulttuuri', 'talous', 'luonto', 'politiikka', 'sää', 'kolumnit', 'tiede', 'oppiminen']


C:\Users\joakim\AppData\Local\Temp\ipykernel_2056\212307127.py:3: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(dataSetPath, sep="␞", nrows=2500)


In [27]:
dataset = Dataset.from_pandas(df, preserve_index=False)
dataset

Dataset({
    features: ['text', 'labels'],
    num_rows: 2496
})

In [28]:

train_test_split = dataset.train_test_split(test_size=0.2)

train_valid_split = train_test_split['train'].train_test_split(test_size=0.1)

datasets = DatasetDict({
    'train': train_valid_split['train'],
    'validation': train_valid_split['test'],
    'test': train_test_split['test']
})
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 1796
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 200
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 500
    })
})


In [29]:
unique_values = set(datasets["train"]["labels"])
print(unique_values)


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}


In [30]:
def tokenize_function(example):
    tokenized = tokenizer(example["text"], truncation=True)
    # tokenized["labels"] = [str2int[label] for label in tokenized["labels"]]
    return tokenized

In [31]:

tokenized_datasets = datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_datasets



Map:   0%|          | 0/1796 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1796
    })
    validation: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 200
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 500
    })
})

In [32]:
print(tokenized_datasets["train"][0]["token_type_ids"])
print(tokenized_datasets["train"][1]["token_type_ids"])
# print(tokenized_datasets["train"][0]["text"])
# print(tokenized_datasets["train"][1]["text"])

# tokenized_datasets["train"][1]
# tokenized_datasets["train"][2]
# tokenized_datasets["train"][3]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [33]:
training_args = TrainingArguments("test")
model = AutoModelForSequenceClassification.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1", num_labels=11, device_map = 'cuda')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

c:\Users\joakim\anaconda3\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


In [35]:
trainer.train()

  0%|          | 0/675 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.4805, 'learning_rate': 1.2962962962962962e-05, 'epoch': 2.22}
{'train_runtime': 1818.0479, 'train_samples_per_second': 2.964, 'train_steps_per_second': 0.371, 'train_loss': 0.40458088627567995, 'epoch': 3.0}


TrainOutput(global_step=675, training_loss=0.40458088627567995, metrics={'train_runtime': 1818.0479, 'train_samples_per_second': 2.964, 'train_steps_per_second': 0.371, 'train_loss': 0.40458088627567995, 'epoch': 3.0})

In [36]:
predictions = trainer.predict(test_dataset=tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)


  0%|          | 0/63 [00:00<?, ?it/s]

(500, 11) (500,)


In [37]:

preds = np.argmax(predictions.predictions, axis=-1)
print(preds)
print(predictions.label_ids)


[ 7  1 10  7 10  1  1  9 10  9  1  9 10  9 10  1  9  9  9  1  1  2  9  7
  7  9  1  1  9  1 10  5  9 10 10  7  7 10 10  1 10  2  9 10 10  9  5  9
 10  7  9 10  9  1  1 10 10 10  9 10  9  1 10 10  1 10  1 10  9  7  1 10
 10  7  1  6 10 10 10 10 10 10 10 10  1  9 10  7  7  7 10  9 10  9  7  1
  7 10 10 10  2 10  9 10  7  7 10 10  7  6  7  9  7  1  2  1 10 10  9  9
  2 10 10 10  1  2  9  7 10 10 10  2 10  1  7  6  9 10 10 10  7  2 10 10
 10 10  1  6  1 10  3 10 10 10 10  9  9  1 10  9 10 10  1  1 10  9  7 10
  1 10  5  9 10  9  9 10 10 10 10  9  2  2  1 10  9  1  9 10 10 10  7  9
  9  7  9 10 10  1  9 10  1 10 10 10  1  9 10  9  1 10  9  9  2 10  7  6
 10 10 10  1  9 10  3  7  7 10 10 10  2  6 10 10 10 10  9 10  7  3  9  1
  7  7  1 10 10 10 10  1  1  1  2  7  6 10  9  9 10 10 10  1  1  9  7  7
 10 10  7 10 10  9  7  9 10  9 10  1  7 10  9  7  1 10  5 10  9 10 10  9
  7  9  1  9  1 10  1  7  9  9  2  1  9 10  1  6  2  9  1 10 10  1 10 10
  1  1  2  1 10 10  7 10  7 10 10  1  1  2  1 10  1

In [38]:
# print(predictions.label_ids)

accuracy = accuracy_score(tokenized_datasets["test"]["labels"], preds)

f1 = f1_score(tokenized_datasets["test"]["labels"], preds, average="micro")

print("Accuracy:", accuracy)
print("F1 Score:", f1)

Accuracy: 0.872
F1 Score: 0.872


In [39]:
trainer.evaluate(tokenized_datasets["test"])



  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.5640786290168762,
 'eval_runtime': 77.7001,
 'eval_samples_per_second': 6.435,
 'eval_steps_per_second': 0.811,
 'epoch': 3.0}